In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load pre-trained CNN model
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

# Get the first convolutional layer
conv_layer = model.get_layer('block1_conv1')

# Pick a filter to visualize
filter_index = 0

# Generate a random image
input_image = np.random.random((1, 224, 224, 3))

# Define a function to calculate the activation of a filter
def filter_activation(input_image, conv_layer, filter_index):
    # Create a new model that outputs the activation of the target filter
    new_model = tf.keras.models.Model(inputs=model.inputs, outputs=conv_layer.output[:, :, :, filter_index])
    # Calculate the activation of the filter
    activation = new_model.predict(input_image)
    return activation

# Define a function to modify the input image to maximize the activation of a filter
def maximize_activation(input_image, conv_layer, filter_index, iterations=20, step_size=1.0):
    for i in range(iterations):
        # Calculate the activation of the filter
        activation = filter_activation(input_image, conv_layer, filter_index)
        # Calculate the gradient of the activation with respect to the input image
        gradient = tf.image.gradients(activation, input_image)[0]
        # Normalize the gradient
        gradient /= (tf.sqrt(tf.reduce_mean(tf.square(gradient))) + 1e-5)
        # Modify the input image to increase the activation of the filter
        input_image += gradient * step_size
    return input_image

# Visualize the filter
activation = filter_activation(input_image, conv_layer, filter_index)
plt.imshow(activation[0, :, :, 0], cmap='gray')
plt.show()

# Visualize the modified input image
modified_image = maximize_activation(input_image, conv_layer, filter_index)
plt.imshow(modified_image[0])
plt.show()